# Assignment 2 - Differential kinematics

## Robot diagram - SCARA robot with YZX spherical wrist 

![](https://github.com/4ku/Inno-robotics-labs/raw/master/Dynamics%20of%20non%20linear%20robotic%20systems/Assignment%201/SCARA_diagram.png)


## Jacobians 
### Geometrical approach

![](https://github.com/4ku/Inno-robotics-labs/raw/master/Dynamics%20of%20non%20linear%20robotic%20systems/Assignment%202/Theory.jpg)

**Example for second joint:**

![](https://github.com/4ku/Inno-robotics-labs/raw/master/Dynamics%20of%20non%20linear%20robotic%20systems/Assignment%202/geometric_example.png)

### Numerical approach

![](https://github.com/4ku/Inno-robotics-labs/raw/master/Dynamics%20of%20non%20linear%20robotic%20systems/Assignment%202/Jacobians_numerical.jpg)

**Example for second joint**

![](https://github.com/4ku/Inno-robotics-labs/raw/master/Dynamics%20of%20non%20linear%20robotic%20systems/Assignment%202/numerical_example.png)

## Implementation

In [1]:
%matplotlib widget
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
np.set_printoptions(suppress=True)

#This class is required for handy matrix multiplication.
class Matrix:
    
    __array_ufunc__ = None
    
    def __init__(self, val):
        self.val = val
        
    def __mul__(self, b):
        if isinstance(b, Matrix):
            val = np.matmul(self.val, b.val)
        else:
            val = np.matmul(self.val, b)
        return Matrix(val)
    
    def __rmul__(self, a):
        if isinstance(a, Matrix):
            val = np.matmul(a.val, self.val)
        else:
            val = np.matmul(a, self.val)
        return Matrix(val)

#Rotation around x axis
class Rx(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[3,3] = 1
        rot[0,0] = 1
        rot[1,:] = [0, np.cos(q), -np.sin(q),0]
        rot[2,:] = [0, np.sin(q), np.cos(q),0]
        self.val = rot

#Partial derivative of rotation x matrix
class dRx(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[1,:] = [0, -np.sin(q), -np.cos(q),0]
        rot[2,:] = [0, np.cos(q), -np.sin(q),0]
        self.val = rot
        
#Rotation around y axis
class Ry(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[3,3] = 1
        rot[:3,:3] = [[np.cos(q), 0, np.sin(q)],
         [0, 1, 0],
         [-np.sin(q), 0, np.cos(q)]
        ]
        self.val = rot

#Partial derivative of rotation y matrix
class dRy(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[:3,:3] = [[-np.sin(q), 0, np.cos(q)],
                      [0, 0, 0],
                      [-np.cos(q), 0, -np.sin(q)]
                     ]
        self.val = rot
        
#Rotation around z axis
class Rz(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[3,3] = 1
        rot[0,:] = [np.cos(q), -np.sin(q),0,0]
        rot[1,:] = [np.sin(q), np.cos(q),0,0]
        rot[2,2] = 1
        self.val = rot

#Partial derivative of rotation z matrix
class dRz(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[0,:] = [-np.sin(q), -np.cos(q),0,0]
        rot[1,:] = [np.cos(q), -np.sin(q),0,0]
        self.val = rot
        
#Transition matrix
class T(Matrix):
    def __init__(self, vector):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[0:3,3] = vector
        np.fill_diagonal(mat,1)  
        self.val = mat

#Transition matrix along x axis
class Tx(Matrix):
    def __init__(self, q):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[0,3] = q
        np.fill_diagonal(mat,1)  
        self.val = mat

#Transition matrix along y axis
class Ty(Matrix):
    def __init__(self, q):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[1,3] = q
        np.fill_diagonal(mat,1)  
        self.val = mat

#Transition matrix along z axis
class Tz(Matrix):
    def __init__(self, q):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[2,3] = q
        np.fill_diagonal(mat,1)  
        self.val = mat

#Partial derivative of transition z matrix
class dTz(Matrix):
    def __init__(self, q):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[2,3] = 1
        self.val = mat

In [2]:
class Jacobians(widgets.HBox):
    
    def extract_vectors_from_trans(trans):
        return trans[:3,3], trans[:3,:3]

    def update_q1(self, q1):
        self.R1 = Rz(q1.new)
        self.dR1 = dRz(q1.new)
        self.update() 
 
    def update_q2(self, q2):
        self.R2 = Rz(q2.new)
        self.dR2 = dRz(q2.new)
        self.update()

    def update_q4(self, q4):
        self.q4 = q4.new
        self.T35 = Tz(-self.L3-q4.new)
        self.dT35 = dTz(-self.L3-q4.new)
        self.update()

    def update_q5(self, q5):
        self.R5 = Ry(q5.new)
        self.dR5 = dRy(q5.new)
        self.update()
        
    def update_q6(self, q6):
        self.R6 = Rz(q6.new)
        self.dR6 = dRz(q6.new)
        self.update()
        
    def update_q7(self, q7):
        self.R7 = Rx(q7.new)
        self.dR7 = dRx(q7.new)
        self.update()
        
    def __init__(self):
        super().__init__()
        
        self.Lbase, self.L1, self.L2, self.L3 = 1, 1, 1, 1 
        q1, q2, q4, q5, q6, q7 = 0, 0, 0, 0, 0, 0
        
        self.Tbase = Tz(self.Lbase)
        self.R1 = Rz(q1)
        self.T12 = Tx(self.L1)
        self.R2 = Rz(q2)
        self.T23 = Tx(self.L2)
        self.T35 = Tz(-self.L3-q4)
        self.R5 = Ry(q5)
        self.R6 = Rz(q6)
        self.R7 = Rx(q7)
        
        self.dR1=dRz(q1)
        self.dR2=dRz(q2)
        self.dT35=dTz(-self.L3-q4)
        self.dR5=dRy(q5)
        self.dR6=dRz(q6)
        self.dR7=dRx(q7)
        
        
        q1_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q1, description='q1')
        q2_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q2, description='q2')
        q4_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q4, description='q4')
        q5_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q5, description='q5')
        q6_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q6, description='q6')
        q7_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q7, description='q7')
        
        q1_slider.observe(self.update_q1, 'value')
        q2_slider.observe(self.update_q2, 'value')
        q4_slider.observe(self.update_q4, 'value')
        q5_slider.observe(self.update_q5, 'value')
        q6_slider.observe(self.update_q6, 'value')
        q7_slider.observe(self.update_q7, 'value')
        
        self.A_res_widget = widgets.Textarea(value='', disabled=True, layout={'height': '400px', 'width': '400px'})
        
        controls = widgets.VBox([
            q1_slider, q2_slider, q4_slider, q5_slider, q6_slider, q7_slider,
        ])
        controls.layout = widgets.Layout(
            display='flex',
            flex='flex-shrink'
         )
        
        result_widget = widgets.VBox([
            self.A_res_widget
        ])
        
        self.children = [controls, result_widget]
        self.update()
    
    
    def forward(self):
        A1 = self.Tbase
        A2 = self.R1 * self.T12
        A3 = self.R2 * self.T23
        A4 = self.T35
        A5 = self.R5
        A6 = self.R6

        A = [A1, A2, A3, A4, A5, A6]
        states = [Matrix(np.eye(4))]

        for a in A:
            states.append(states[-1] * a)
        
        self.A_res = states[-1] * self.R7
        return states[1:]
    

    def get_geometrical_jacobian(self, states):
        On, rotations = Jacobians.extract_vectors_from_trans(states[-1].val)

        O0, rot0 = Jacobians.extract_vectors_from_trans(states[0].val)
        U0 = rot0[:, 2]
        jp = np.cross(U0, On-O0)
        J1 = np.concatenate((jp,U0))

        O1, rot1 = Jacobians.extract_vectors_from_trans(states[1].val)
        U1 = rot1[:, 2]
        jp = np.cross(U1, On-O1)
        J2 = np.concatenate((jp,U1))

        O2, rot2 = Jacobians.extract_vectors_from_trans(states[2].val)
        U2 = rot2[:, 2]
        J3 = np.concatenate((U2,np.zeros(3)))

        O3, rot3 = Jacobians.extract_vectors_from_trans(states[3].val)
        U3 = rot3[:, 1]
        jp = np.cross(U3, On-O3)
        J4 = np.concatenate((jp,U3))

        O4, rot4 = Jacobians.extract_vectors_from_trans(states[4].val)
        U4 = rot4[:, 2]
        jp = np.cross(U4, On-O4)
        J5 = np.concatenate((jp,U4))

        O5, rot5 = Jacobians.extract_vectors_from_trans(states[5].val)
        U5 = rot5[:, 0]
        jp = np.cross(U5, On-O5)
        J6 = np.concatenate((jp,U5))
        
        J = np.column_stack((J1,J2,J3,J4,J5,J6))
        return J
    
    
    def get_numerical_jacobian(self):
        R_inv = np.eye(4)
        R_inv[:3,:3] = np.linalg.inv(self.A_res.val[:3,:3])
        
        
        jr1 =  (self.Tbase * self.dR1 * self.T12 * self.R2 * \
                    self.T23 * self.T35 * self.R5 * self.R6 * self.R7 * R_inv).val
        J1 = np.concatenate((jr1[:3,3],[jr1[2,1],jr1[0,2],jr1[1,0]]))
        
        
        jr2 =  (self.Tbase * self.R1 * self.T12 * self.dR2 * \
                    self.T23 * self.T35 * self.R5 * self.R6 * self.R7 * R_inv).val
        J2 = np.concatenate((jr2[:3,3],[jr2[2,1],jr2[0,2],jr2[1,0]]))
        
        
        jp =  (self.Tbase * self.R1 * self.T12 * self.R2 * \
                    self.T23 * self.dT35 * self.R5 * self.R6 * self.R7).val
        J3 = np.concatenate((jp[:3,3],np.zeros(3)))
        
        
        jr4 =  (self.Tbase * self.R1 * self.T12 * self.R2 * \
                    self.T23 * self.T35 * self.dR5 * self.R6 * self.R7 * R_inv).val
        J4 = np.concatenate((jr4[:3,3],[jr4[2,1],jr4[0,2],jr4[1,0]]))
        
        
        jr5 =  (self.Tbase * self.R1 * self.T12 * self.R2 * \
                    self.T23 * self.T35 * self.R5 * self.dR6 * self.R7 * R_inv).val
        J5 = np.concatenate((jr5[:3,3],[jr5[2,1],jr5[0,2],jr5[1,0]]))

        
        jr6 =  (self.Tbase * self.R1 * self.T12 * self.R2 * \
                    self.T23 * self.T35 * self.R5 * self.R6 * self.dR7 * R_inv).val
        J6 = np.concatenate((jr6[:3,3],[jr6[2,1],jr6[0,2],jr6[1,0]]))
        
        
        J = np.column_stack((J1,J2,J3,J4,J5,J6))
        return J


    def update(self):
        states = self.forward()
        J_num = self.get_numerical_jacobian()
        J_geom = self.get_geometrical_jacobian(states)
        
        result_message = "Jacobian matrix (geometrical approach):\n" + np.array2string(J_geom, precision=2) + "\n\n"
        result_message += "Jacobian matrix (numerical approach):\n" + np.array2string(J_num, precision=2) + "\n\n"
        result_message += "Result matrix:\n" + np.array2string(self.A_res.val, precision=2) + "\n\n"
        self.A_res_widget.value = result_message
        
Jacobians()

Jacobians(children=(VBox(children=(FloatSlider(value=0.0, description='q1', max=6.283185307179586, step=0.01),…